This notebook objective is Multi-tissue prediction of mRNA half-life from sequence using transfer learning of Hybird RNN-CNN with Batch Normalization model that was trained on general half-life prediction on Saluki dataset

## Environment Setup

In [ ]:
import pandas as pd
from kipoiseq.transforms.functional import one_hot, fixed_len
import numpy as np
from plotnine import ggplot, aes, geom_histogram


from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.models import Model
import tensorflow_addons as tfa
import tensorflow as tf
import keras.layers as kl
import keras
import numpy as np
# we use the kipoiseq package for one hot encoding
from kipoiseq.transforms.functional import one_hot, fixed_len

## Data Preprocessing
- Reading Multi-tissue Dataset
- one-hot encode of 6 tracks
    - (4) RNA sequence (A, G, T, C)
    - (1) Exon binding sites
    - (1) starts of codons
    

In [ ]:
# Read dataset exercise dataset with mean_hl and exons junctions
# NaNs for tissues hl were set to -1000.0
tissue_hl = pd.read_csv('../data/genomic_sequence_plus_features_hl_all_tissues_with_ss.csv', index_col=0)
tissue_hl

In [ ]:
columns = tissue_hl.columns[1:]
columns

In [ ]:
# drop nans
# keep mRNAs with annotated 3' and 5' UTRs
tissue_hl = tissue_hl.loc[tissue_hl.loc[:, ['3_utr', '5_utr']].dropna().index]
tissue_hl

In [ ]:
tissue_hl.shape

In [ ]:
seqs = tissue_hl['5_utr'] + tissue_hl['cds'] + tissue_hl['3_utr'] #full sequence
print(seqs)
len_seqs = pd.DataFrame([len(seq) for seq in seqs], columns=['len'])

(ggplot(len_seqs, aes('len'))
  + geom_histogram())

In [ ]:
# Cuttoff 10000 nucleotides
max_len = 10000
def pad_sequence(seqs, max_len, anchor='start', value='N'):
  padded_seqs = [fixed_len(seq, max_len, anchor=anchor) for seq in seqs.astype("string")]
  return padded_seqs
fixed_len_seqs = np.array(pad_sequence(seqs, max_len))
del seqs

In [ ]:
#track six
starts = []

for i in range(len(tissue_hl)):
  #assert len(sequences['ORF_seqs'].astype("string")[i]) % 3 == 0 
    lst = list(range(len(tissue_hl['cds'].astype("string")[i])))
    onehot = np.repeat(0, repeats = len(tissue_hl['cds'].astype("string")[i]))

    onehot[lst[0::3]] = 1
    full = np.concatenate((np.repeat([0], repeats = len(tissue_hl['5_utr'].astype("string")[i])),
                         onehot,
                         np.repeat([0], repeats = len(tissue_hl['3_utr'].astype("string")[i]))), axis=None)
    
    if (len(full) > max_len):
        full = full[:max_len]
    elif (len(full) < max_len):
        full = np.concatenate((full, np.repeat(0, repeats = max_len - len(full))),axis = None)
    starts.append(full)

In [ ]:
# one hot for track 5: the exon binding sites
exons = []

for i in range(len(tissue_hl)):
  onehot = np.repeat(0, repeats = max_len)
  if(isinstance(tissue_hl["Exon_Junctions"][i], str)):
    current_exons = list(map(int, tissue_hl["Exon_Junctions"][i].split(";")))
    assert len(current_exons) > 0
    positions_capped = [x for x in current_exons if x <= 10000] # delete all exon junctions after 10000 since we're capping the sequence there
    onehot[positions_capped] = 1
  exons.append(onehot)

In [ ]:
one_hot_seqs = np.array([one_hot(seq, neutral_value=0) for seq in fixed_len_seqs])
seqs = np.stack([one_hot_seqs[:,:,0], one_hot_seqs[:,:,1], one_hot_seqs[:,:,2], one_hot_seqs[:,:,3],
                 np.array(exons), np.array(starts)], axis = 2)

In [ ]:
seqs.shape

In [ ]:
del [starts, full, len_seqs, one_hot_seqs]

## Prepare dataset for training
split according to chromosomes

In [ ]:
chrom_val = ['chr2', 'chr3', 'chr4']
chrom_test = ['chr1', 'chr8', 'chr9']

In [ ]:
idx_test = np.where(tissue_hl.chromosome.isin(chrom_test))[0]
idx_val = np.where(tissue_hl.chromosome.isin(chrom_val))[0]
idx_train = np.where(~(tissue_hl.chromosome.isin(chrom_test)| tissue_hl.chromosome.isin(chrom_val)))[0]

In [ ]:
def train_test_split(array, idx_train, idx_val, idx_test):
  return array[idx_train], array[idx_val], array[idx_test]

In [ ]:
tissues = tissue_hl.columns[1:50]
tissues

In [ ]:
tissues.shape

In [ ]:
# Mask nans with -1000
mask_value = -1000
tissue_hl.loc[:, tissues] = tissue_hl.loc[:, tissues].fillna(mask_value)

In [ ]:
X_train, X_val, X_test = train_test_split(seqs, idx_train, idx_val, idx_test)
y_vars = list(tissues)
y_train, y_val, y_test = train_test_split(tissue_hl.loc[:, y_vars].values, idx_train, idx_val, idx_test)

In [ ]:
y_train

## Training the multi-task model

In [ ]:
# Upload Hybird CNN-RNN with Batch Normalization model that was trained on Saluki dataset
model = keras.models.load_model('../data/Models for transfer learning/my_saluki5_23_v2.h5')

### Model transfer and last layers change 

In [ ]:
## Create a new model by using part of the model constructed previously by keeping all layers but the last one.

transferred_model = Model(model.inputs, model.layers[-2].output)

transferred_model.summary()

In [ ]:
# Freeze layers
for layer in transferred_model.layers:
  layer.trainable = False

In [ ]:
# Create the multi-task model by using the transferred layers plus extra layers for the prediction on each task

input = kl.Input(X_train.shape[1:])
x = transferred_model(input)

x = kl.Dense(units = 128, kernel_regularizer=tf.keras.regularizers.l2(l=0.0015))(x)
x = kl.Dense(units = 128, kernel_regularizer=tf.keras.regularizers.l2(l=0.0015))(x)
x = tfa.layers.GELU()(x)
print(x.shape)

output = kl.Dense(units=len(tissues))(x)
mt_model_transfer = Model(inputs=input, outputs=output)
# mt_model_transfer.summary()

In [ ]:
# Define custom loss functions to handle NaNs
from keras import backend as K
mask_value = -1000
def function_masked_mse(y_true, y_pred):
        mask = K.cast(K.not_equal(y_true, mask_value), K.floatx())
        masked_summed_error = K.sum(K.square(mask * (y_true - y_pred)), axis=1)
        smooth=0
        masked_mean_squared_error = masked_summed_error / (K.sum(mask, axis=1) + smooth)

        return masked_mean_squared_error

In [ ]:
# We now compile the model using our custom loss function: function_masked_mse, use lw lr
mt_model_transfer.compile(optimizer=keras.optimizers.Adam(lr = 0.0001), loss=function_masked_mse)

# Train the model
history_transfer = mt_model_transfer.fit(X_train, 
                    y_train, 
                    validation_data=(X_val, y_val),
                    callbacks=[EarlyStopping(patience=15, restore_best_weights=True),
                              ],
                    batch_size=32,  
                    epochs=300)

In [ ]:
# Define function to plot val and train losses
import matplotlib.pyplot as plt
def plot_loss(history):
    fig, ax = plt.subplots(figsize = (5,5))
    ax.plot(history['loss'][1:], label="train_loss")
    ax.plot(history['val_loss'][1:], label="val_loss")
    plt.xlabel('epoch')
    plt.legend()
    plt.ylabel('mean squared error')

In [ ]:
plot_loss(history_transfer.history)

### Unfreeze some layers and re-train

In [ ]:
mt_model_transfer.get_layer('model').summary()

In [ ]:
# unfreeze last dense layer
mt_model_transfer.get_layer('model').get_layer('dense').trainable = True


In [ ]:
mt_model_transfer.compile(optimizer=keras.optimizers.Adam(lr = 0.0001), loss=function_masked_mse)

# Train the model
history_transfer = mt_model_transfer.fit(X_train, 
                    y_train, 
                    validation_data=(X_val, y_val),
                    callbacks=[EarlyStopping(patience=15, restore_best_weights=True),
                    ],
                    batch_size=32,  
                    epochs=1000)

In [ ]:
plot_loss(history_transfer.history)

In [ ]:
# Save model
mt_model_transfer.save("model_exp_2")

Now let's evaluate the final model on each task/tissue.

In [ ]:
preds_transfer_val = mt_model_transfer.predict(X_val)

preds_transfer_val_df = pd.DataFrame(preds_transfer_val, columns=y_vars, 
                                     index=tissue_hl.iloc[idx_val].index)

true_val_df = tissue_hl.iloc[idx_val].loc[:, y_vars]

In [ ]:
import plotnine as p9

# This function inputs 2 dataframes with tasks as columns and 
# mRNAs as rows. One data frame contains the true (measured) values
# and the other the predicted ones.
# The output is a dataframe with 
# 2 columns: task and explained variance score

def get_scores(true_df, pred_df):

  exp_var_scores = []
  r2_scores = []

  for y_var in y_vars:
    non_na_idxs = true_df[true_df[y_var]!= mask_value].index
    exp_var_scores.append(explained_variance_score(true_df.loc[non_na_idxs, y_var].values, pred_df.loc[non_na_idxs, y_var].values))
    r2_scores.append(r2_score(true_df.loc[non_na_idxs, y_var].values, pred_df.loc[non_na_idxs, y_var].values))
  scores_df = pd.DataFrame({'task':y_vars, 'exp_var_score': exp_var_scores, "r2_score": r2_scores})

  return scores_df




In [ ]:
# Get the scores
val_transfer_scores_df = get_scores(true_val_df, preds_transfer_val_df)

# Plot scores per task
fig = p9.ggplot(val_transfer_scores_df, p9.aes('task', 'exp_var_score')) + p9.geom_col() + p9.theme(axis_text_x = p9.element_text(angle = 90))
fig

In [ ]:
var_score = explained_variance_score(true_val_df, preds_transfer_val_df)

### Test our model

In [ ]:
preds_transfer_test = mt_model_transfer.predict(X_test)
preds_transfer_test_df = pd.DataFrame(preds_transfer_test, columns=y_vars, 
                                     index=tissue_hl.iloc[idx_test].index)

true_test_df = tissue_hl.iloc[idx_test].loc[:, y_vars]
# Get the scores
test_transfer_scores_df = get_scores(true_test_df, preds_transfer_test_df)

# Plot scores per task
p9.ggplot(test_transfer_scores_df, p9.aes('task', 'exp_var_score')) + p9.geom_col() + p9.theme(axis_text_x = p9.element_text(angle = 90))
fig

### All layers are trainable

In [ ]:
for layer in transferred_model.layers:
  layer.trainable = True

In [ ]:
## Now create the multi-task model by using the transferred layers plus extra layers for the prediction on each task

input = kl.Input(X_train.shape[1:])
x = transferred_model(input)



x = kl.Dense(units = 128)(x)
x = kl.Dense(units = 128)(x)
x = tfa.layers.GELU()(x)
output = kl.Dense(units=len(tissues))(x)
mt_model_transfer_scratch = Model(inputs=input, outputs=output)
# mt_model_transfer_scratch.summary()

In [ ]:
# We now compile the model using our custom loss function: function_masked_mse
mt_model_transfer_scratch.compile(optimizer=keras.optimizers.Adam(lr = 0.0001), loss=function_masked_mse)

# Train the model
history_transfer_scratch = mt_model_transfer_scratch.fit(X_train, 
                    y_train, 
                    validation_data=(X_val, y_val),
                    callbacks=[EarlyStopping(patience=15, restore_best_weights=True),
                    ],
                    batch_size=32,  
                    epochs=1000)

In [ ]:
plot_loss(history_transfer_scratch.history)

In [ ]:
# Save model
mt_model_transfer_scratch.save("model_exp_3.h5")

#### Test our model

In [ ]:
# test in val data
preds_transfer_val = mt_model_transfer_scratch.predict(X_val)

preds_transfer_val_df = pd.DataFrame(preds_transfer_val, columns=y_vars, 
                                     index=tissue_hl.iloc[idx_val].index)

true_val_df = tissue_hl.iloc[idx_val].loc[:, y_vars]
# Get the scores
val_transfer_scores_df = get_scores(true_val_df, preds_transfer_val_df)

# Plot scores per task
fig = p9.ggplot(val_transfer_scores_df, p9.aes('task', 'exp_var_score')) + p9.geom_col() + p9.theme(axis_text_x = p9.element_text(angle = 90))
fig

In [ ]:
# test in val data
preds_transfer_test = mt_model_transfer_scratch.predict(X_test)
preds_transfer_test_df = pd.DataFrame(preds_transfer_test, columns=y_vars, 
                                     index=tissue_hl.iloc[idx_test].index)

true_test_df = tissue_hl.iloc[idx_test].loc[:, y_vars]
# Get the scores
test_transfer_scores_df = get_scores(true_test_df, preds_transfer_test_df)

# Plot scores per task
fig = p9.ggplot(test_transfer_scores_df, p9.aes('task', 'exp_var_score')) + p9.geom_col() + p9.theme(axis_text_x = p9.element_text(angle = 90))
fig

#### Analyse Results

In [ ]:
# Explained-variance score stats across tissues
print("min: ", test_transfer_scores_df["exp_var_score"].min())
print("max: ", test_transfer_scores_df["exp_var_score"].max())
print("mean: ", test_transfer_scores_df["exp_var_score"].mean())
print("median: ", test_transfer_scores_df["exp_var_score"].median())

In [ ]:
# Get tissues with top 4 explained-variance
test_transfer_scores_df.sort_values(["exp_var_score"], ascending=False)[:4]